Importando bibliotecas necessárias

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

Carregando dataset

In [3]:
dataset_url = 'https://drive.usercontent.google.com/download?id=11ijjHpIWkZq8_M5ke8VWYvojOhXq1eJU&export=download'
dataset = pd.read_csv(dataset_url, delimiter=';')

Definindo constantes dos headers dos dados

In [4]:
pais_header = 'Pais'
cod_pais_header = 'CodPais'
ano_header = 'Ano'
regime_politico_header = 'Regime Politico'

### Visualização 1

In [5]:
regimes_por_ano = dataset[[ano_header, regime_politico_header]]

cores = {
    '0 - Autocracia Fechada': '#FFD700',
    '1 - Autocracia Eleitoral': '#1E90FF',
    '2 - Democracia Eleitoral': '#7CFC00',
    '3 - Democracia Liberal': '#FF4500'
}

regimes_por_ano['cor'] = regimes_por_ano[regime_politico_header].map(cores)

fig = px.parallel_categories(
    regimes_por_ano,
    dimensions=[regime_politico_header, ano_header],
    color='cor'
)

fig.update_layout(title_text="Dinâmica de transformação de regimes políticos no mundo")

fig.show()

<ipython-input-5-6435dd40a955>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regimes_por_ano['cor'] = regimes_por_ano[regime_politico_header].map(cores)


### Visualização 2

In [6]:
mudancas_header = 'Mudancas'

def contar_mudancas_regime(pais_mudancas):
    pais_mudancas = pais_mudancas.sort_values(ano_header)
    qtd_mudancas = (pais_mudancas[regime_politico_header] != pais_mudancas[regime_politico_header].shift(1)).sum() - 1
    return qtd_mudancas

qtd_mudancas_por_pais = dataset.groupby(cod_pais_header).apply(contar_mudancas_regime).reset_index()
qtd_mudancas_por_pais.columns = [cod_pais_header, mudancas_header]

print(qtd_mudancas_por_pais.sort_values(ascending=False, by=mudancas_header).head())

    CodPais  Mudancas
90      LKA         2
160     THA         2
100     MKD         2
65      HND         2
26      CHL         2


In [7]:
escala_de_cor = [
    [0, 'white'], # 0 mudanças
    [1/2, '#c6dbf0'], # 1 mudança
    [1, '#6bafd6'] # 2 mudanças
]

titulo_grafico = "Mapa de Oscilação Política: Países com Mais Mudanças de Regime de 1980 a 2020."
projection = 'natural earth'

fig = px.choropleth(qtd_mudancas_por_pais,
                    locations=cod_pais_header,
                    color=mudancas_header,
                    hover_name=cod_pais_header,
                    color_continuous_scale=escala_de_cor,
                    projection=projection,
                    title=titulo_grafico)

fig.update_layout(coloraxis_colorbar=dict(
    tickvals=[0, 1, 2],
    ticktext=['Não houve alteração de regime político', '1 Alteração de regime político', '2 Alterações de regime político']
))

fig.show()